## Deep Watching


In [ ]:
from collections import defaultdict
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
import cv2
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from sklearn.manifold import Isomap

In [ ]:
# input file
FILE = "results.pkl"
with open(FILE, 'rb') as fi:
    results = pickle.load(fi)
    
# available classes
classes = results['classes']

In [ ]:
# load class info from pickle
found_classes = []
scores = []
areas = []
centers_x = []
centers_y = []
boxes = []
paths = []
for path in results['instances']:
    img_size = results['instances'][path].image_size
    found_classes.extend([classes[x] for x in results['instances'][path].pred_classes.numpy()])
    scores.extend(results['instances'][path].scores.numpy())
    areas.extend([x / (img_size[0] * img_size[1]) for x in results['instances'][path].pred_boxes.area().numpy()])
    centers_x.extend([x[0] / img_size[1] for x in results['instances'][path].pred_boxes.get_centers().numpy()])
    centers_y.extend([x[1] / img_size[0] for x in results['instances'][path].pred_boxes.get_centers().numpy()])
    boxes.extend(results['instances'][path].pred_boxes.tensor.numpy())
    paths.extend([path] * len(results['instances'][path].pred_boxes))
data = pd.DataFrame(list(zip(paths, found_classes, scores, areas, centers_x, centers_y, boxes)), columns = ['path', 'found_class', 'score', 'area', 'center_x', 'center_y', 'box']) 
data

In [ ]:
found_classes = data.groupby('found_class').agg({'box':'size','score':'mean', 'area':'mean', 'center_x':'mean', 'center_y':'mean'}).rename(columns={'box':'count'}).reset_index().sort_values(by=['count'], ascending=False)
found_classes

In [ ]:
classes_per_image = data.groupby(['path', 'found_class'])['box'].count().unstack().fillna(0)
classes_per_image

# Corpus Map using Class Counts

In [ ]:
matrix = classes_per_image.to_numpy()

# create isomap
imap = Isomap()
corpus_map = imap.fit_transform(matrix)

# plot corpus using isomap
label_scale=0.1
fig = plt.figure(figsize=(18, 12))
ax1 = fig.add_subplot(111)
for counter in range(len(corpus_map)):
    scatter = ax1.scatter(corpus_map[counter][0], corpus_map[counter][1], marker='o', s=600)
    plt.annotate(classes_per_image.index.values[counter], xy=(corpus_map[counter][0]+label_scale, corpus_map[counter][1]+label_scale))
plt.show()

# Symbol cooccurrences

In [ ]:
import matplotlib.colors as clrs
c = classes_per_image.columns.values
cooc = np.zeros(shape=(len(c), len(c)))
for row in matrix:
    for index_a, a in enumerate(row):
        for index_b, b in enumerate(row):
            if a == b and a > 0:
                cooc[index_a][index_b] += a-1
            if a != b and a > 0 and b > 0:
                cooc[index_a][index_b] = a
                cooc[index_b][index_a] = b
                   
fig, ax = plt.subplots(figsize=(18,10))
ax.grid(False)
plt.imshow(cooc, norm=clrs.LogNorm(vmin=0.1, vmax=cooc.max()),)
plt.colorbar()
plt.xticks(range(len(c)), c, rotation=90)
plt.yticks(range(len(c)), c)
plt.show()